In [136]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2024-12-19 07:22:38--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Loaded CA certificate '/etc/ssl/certs/ca-certificates.crt'
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt.1’

input.txt.1         100%[===================>]   1.06M  1.47MB/s    in 0.7s    

2024-12-19 07:22:39 (1.47 MB/s) - ‘input.txt.1’ saved [1115394/1115394]



In [137]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [138]:
print("length of dataset in characters : ", len(text))

length of dataset in characters :  1115394


In [139]:
print(text[:500])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor


In [140]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [141]:
# create a mapping from char to int
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda i: ''.join(itos[c] for c in i)

print(encode('hi there'))
print(decode(encode('hi there')))

[46, 47, 1, 58, 46, 43, 56, 43]
hi there


In [142]:
# encoding the whole dataset and storing it in torch.tensor
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [143]:
# splitting the data into train and validation sets
n = int(0.9 * len(data))
train_data = data[:n]
val_data = data[n:]

In [144]:
block_size = 8
X = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = X[:t+1]
    target = y[t]
    print(f"when the input is {context} the target : {target}")

when the input is tensor([18]) the target : 47
when the input is tensor([18, 47]) the target : 56
when the input is tensor([18, 47, 56]) the target : 57
when the input is tensor([18, 47, 56, 57]) the target : 58
when the input is tensor([18, 47, 56, 57, 58]) the target : 1
when the input is tensor([18, 47, 56, 57, 58,  1]) the target : 15
when the input is tensor([18, 47, 56, 57, 58,  1, 15]) the target : 47
when the input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target : 58


In [145]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(split:str):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        print(f"b:{b}, t:{t}")
        target = yb[b,t]
        print(f"when input is {context} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
b:0, t:0
when input is tensor([24]) the target: 43
b:0, t:1
when input is tensor([24, 43]) the target: 58
b:0, t:2
when input is tensor([24, 43, 58]) the target: 5
b:0, t:3
when input is tensor([24, 43, 58,  5]) the target: 57
b:0, t:4
when input is tensor([24, 43, 58,  5, 57]) the target: 1
b:0, t:5
when input is tensor([24, 43, 58,  5, 57,  1]) the target: 46
b:0, t:6
when input is tensor([24, 43, 58,  5, 57,  1, 46]) the target: 43
b:0, t:7
when input is tensor([24, 43, 58,  5, 57,  1, 46, 43]) the target: 39
b:1, t:0
when input is tensor([44]) the target: 53
b:1, t:1
when input is tensor([44

In [146]:
# Bi-gram language model
import torch.nn as nn
import torch.nn.functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits from the lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        # (B,T,C) = (Batch(4), Time(8), Channels(65))
        # idx and targets are (B, T) tensors
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets == None:
            loss = None
        else:
            # need to change shape of logits to pass it into loss, (b,t,c) -> (b*t, c)
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply the softmax to get probs
            probs = F.softmax(logits, dim=1) # (B,C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append generated index to the running seqeuence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel(vocab_size)
logits, loss = model(xb, yb)
print(logits.shape)
print(loss)

print(decode(model.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [147]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [148]:
batch_size = 32
for steps in range(1000):
    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

3.7218432426452637


In [149]:
print(decode(model.generate(torch.zeros((1,1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


olylvLLko'TMyatyIoconxad.?-tNSqYPsx&bF.oiR;BD$dZBMZv'K f bRSmIKptRPly:AUC&$zLK,qUEy&Ay;ZxjKVhmrdagC-


# Mathematical trick in self attention

In [150]:
torch.manual_seed(1337)
B,T,C = 4,8,2
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [151]:
# version-1
# we want x[b,t] = mean(i<=t) x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] # (t, C)
        xbow[b,t] = torch.mean(xprev, 0)

xprev.shape, xbow.shape

(torch.Size([8, 2]), torch.Size([4, 8, 2]))

In [152]:
z = torch.tensor([[1,2,3],
    [4,5,6]], dtype=torch.float)
z.mean(1)

tensor([2., 5.])

In [153]:
# version-2
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
# wei will be transformed from (T,T) to (B,T,T) while matmul
xbow2 = wei @ x # (B,T,T) @ (B,T,C) --> (B,T,C)
xbow2.shape, torch.allclose(xbow, xbow2, atol=1e-8)

(torch.Size([4, 8, 2]), False)

In [154]:
tril = torch.tril(torch.ones(T,T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril==0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
xbow3.shape, torch.allclose(xbow2, xbow3)

(torch.Size([4, 8, 2]), True)

In [155]:
wei

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]])

In [156]:
xbow.sum(), xbow2.sum()

(tensor(9.1398), tensor(9.1398))

In [157]:
import torch

# Check exact equality
print("Exact Equality:", torch.equal(xbow, xbow2))

# Check if tensors are close enough (with tolerance for floating-point precision)
print("Close Enough:", torch.allclose(xbow, xbow2, atol=1e-8))

# Check the dtype and device of both tensors
print("xbow dtype:", xbow.dtype, "xbow2 dtype:", xbow2.dtype)
print("xbow device:", xbow.device, "xbow2 device:", xbow2.device)

# Check for NaN or Inf values
print("xbow contains NaN:", torch.isnan(xbow).any(), "xbow contains Inf:", torch.isinf(xbow).any())
print("xbow2 contains NaN:", torch.isnan(xbow2).any(), "xbow2 contains Inf:", torch.isinf(xbow2).any())


Exact Equality: False
Close Enough: False
xbow dtype: torch.float32 xbow2 dtype: torch.float32
xbow device: cpu xbow2 device: cpu
xbow contains NaN: tensor(False) xbow contains Inf: tensor(False)
xbow2 contains NaN: tensor(False) xbow2 contains Inf: tensor(False)


In [158]:
xbow.shape, xprev.shape

(torch.Size([4, 8, 2]), torch.Size([8, 2]))

In [159]:
torch.manual_seed(42)
a = torch.tril(torch.ones(3,3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0, 10, (3,2)).float()
c = a @ b
print('a=')
print(a)
print('---')
print('b=')
print(b)
print('---')
print('c=')
print(c)
print('---')

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
---
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
---
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])
---


In [160]:
# version-4 self-attention
torch.manual_seed(1337)
B, T, C = 4, 8, 32
x = torch.randn(B,T,C)

# single head of self-attention
head_size = 16
query = nn.Linear(C, head_size, bias=False)
key = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
q = query(x) # (B,T,16)
k = key(x)   # (B,T,16)
v = value(x)

wei = q @ k.transpose(-1, -2) # (B, T, 16) * (B, 16, T) --> (B, T, T)
tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril==0, float('-inf'))
wei = F.softmax(wei, dim=1)
# out = wei @ x
out = wei @ v

out.shape, v.shape, wei.shape

(torch.Size([4, 8, 16]), torch.Size([4, 8, 16]), torch.Size([4, 8, 8]))

In [167]:
class BatchNorm1d:
  
  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)
      
  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

torch.manual_seed(1337)
module = BatchNorm1d(100)
x = torch.randn(32, 100)
x = module(x)
x.shape

torch.Size([32, 100])

In [168]:
x[:,0].mean(), x[:,0].std()

(tensor(0.1469), tensor(0.8803))

In [169]:
x[0,:].mean(), x[0,:].std()

(tensor(-9.5367e-09), tensor(1.0000))